In [2]:
import sys
sys.executable

# [directory] -m pip install [pandas, sklearn, keras, tensorflow]

'/Users/jiyeon/anaconda3/envs/datascience/bin/python'

Data Cleaning

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train_raw = pd.read_csv("celestial_train.csv")
test_raw = pd.read_csv("celestial_test.csv", index_col = 0)

train_raw["class"].value_counts() # galaxy is about 3/5 of data; need weight balance

features_columns = ["alpha", "delta", "u", "g", "r", "i", "z", "redshift"] # excluding unrealted info.
train = pd.concat([train_raw[features_columns], train_raw["class"]], axis = 1)
test = test_raw[features_columns].copy()

train["class"] = train["class"].map({"GALAXY": 0, "QSO": 1, "STAR": 2})

train_raw

,id,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,redshift,plate,MJD,fiber_ID,class
0,0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,0.634794,5812,56354,171,GALAXY
1,1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,0.779136,10445,58158,427,GALAXY
2,2,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,0.932346,9149,58039,775,GALAXY
3,3,340.995121,20.589476,23.48827,23.33776,21.32195,20.25615,19.54544,8102,301,3,110,5.658977e+18,1.424659,5026,55855,741,QSO
4,4,200.290475,47.199402,24.40286,22.35669,20.61032,19.46490,18.95852,3716,301,5,108,7.459285e+18,0.660012,6625,56386,719,GALAXY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49995,20.029463,11.414399,22.30954,23.98841,21.82823,20.49596,19.59701,7787,301,2,333,1.246831e+19,0.612791,11074,58488,339,GALAXY
49996,49996,19.286780,15.951587,22.74555,23.01785,21.04041,19.86370,19.25856,7713,301,1,354,5.786070e+18,0.602187,5139,55866,257,GALAXY
49997,49997,146.898055,27.818094,26.22899,21.84820,20.47138,19.60049,18.94726,4569,301,1,234,6.523611e+18,0.550025,5794,56282,535,GALAXY
49998,49998,317.246996,-0.682254,20.96526,19.81625,19.34186,19.14711,19.05790,4187,301,2,64,1.154061e+18,0.175206,1025,53239,51,GALAXY


Preprocessing

In [6]:
features = train[features_columns]
target = train["class"]

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.20)

from sklearn.preprocessing import StandardScaler
scaling = StandardScaler()
X_train = scaling.fit_transform(X_train) # helps fitting data into the model
X_test = scaling.transform(X_test)
test = scaling.transform(test)

Training: Neural Network

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras

model = Sequential()

model.add(Dense(units = 32, activation = "relu", input_dim = 8)) # input layer

for i in range(10):
    model.add(Dense(units = 32, activation = "relu")) # hidden layers
    model.add(Dropout(0.3)) # arbitrarily chosen btw 0.1 ~ 0.5

model.add(Dense(units = 3, activation = "softmax")) # sigmoid is for binary classification, so alternative is used.

opt = keras.optimizers.Adam(learning_rate = 0.001, weight_decay = 1e-6) # weight decay to prevent overfitting
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"]) # have not understood fully...
model.fit(X_train, Y_train, batch_size = 10, epochs = 10, validation_data = (X_test, Y_test))

2024-01-18 20:15:05.137639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jiyeon/anaconda3/envs/datascience/lib/python3.11/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.6725 - loss: 0.7815 - val_accuracy: 0.8483 - val_loss: 0.3537
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9078 - loss: 0.3001 - val_accuracy: 0.9311 - val_loss: 0.2587
Epoch 3/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9335 - loss: 0.2392 - val_accuracy: 0.9537 - val_loss: 0.2214
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9414 - loss: 0.2159 - val_accuracy: 0.9621 - val_loss: 0.1890
Epoch 5/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9455 - loss: 0.2529 - val_accuracy: 0.9530 - val_loss: 0.1605
Epoch 6/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.9468 - loss: 0.2027 - val_accuracy: 0.9481 - val_loss: 0.1911
Epoch 7/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.9496 - loss: 0.1838 - val_accuracy: 0.9622 - val_loss: 0.1443
Epoch 8/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9495 - loss: 0

In [29]:
import numpy as np

Y_prediction = model.predict(test) # returns the probability for each category
output = np.argmax(Y_prediction, axis = 1) # converting probability to actual value

ids = test_raw.index.to_list()
result = pd.DataFrame({"id": ids, "class": output})
result["class"] = result["class"].map({0: "GALAXY", 1: "QSO", 2: "STAR"})
result.to_csv("NN_output.csv", index = False)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


50000